```
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
```

In [ ]:
from google.colab import drive
drive_root = "/content/drive"
drive.mount(drive_root, force_remount=True)

In [ ]:
import multiprocessing as mp
from time import sleep

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
my_drive = GoogleDrive(gauth)

def cleanDriveTrash () :
  for f in my_drive.ListFile({'q': "trashed = true"}).GetList():
    name = f.attr['metadata']['title']
    if len(name) > 4 and (name[-5:] == ".fits" or name[-4:] == ".bz2") :
      try :
        f.Delete()
      except :
        pass

In [ ]:
import os

reldir = 'DAGN' #@param {type:"string"}
try :
  os.chdir(drive_root + "/My Drive" + "/" + reldir)
  print ("Loaded source directory")
except Exception as e :
  print ("Error in loading source directory")
  print ("Please check exception message for details - \n{}".format(e))

In [ ]:
import batch
Batch = batch.Batch
print("Loaded Batch module")

In [ ]:
batch_name = '' #@param {type:"string"}
batchObj = Batch.getBatch(batch_name)

In [ ]:
for i, args in enumerate(batchObj.galaxies) :
    if i % 5 == 0 :
      p = mp.Process(target=cleanDriveTrash)
      p.start()

    csvLine, progressLine = batchObj.classifyGal(args)
    batchObj.reslog.info(csvLine)
    print(progressLine)

In [ ]:
del batchObj
del batch

! rm -rf Batches/$batch_name/FITS
! rm -rf Batches/$batch_name/Results
! rm -rf *.log

i = 1
while i <= 9999999 :
  print("Monitoring trash {}".format(i))
  cleanDriveTrash()
  sleep(60)
  i += 1